In [ ]:
# -*- coding: utf-8 -*-
"""
This script fetches the reviewed UniProt accession IDs for the 48 known
human nuclear receptors. It queries the UniProt API using the official
HGNC gene symbols and saves the results to a file.

Dependencies:
- requests: A Python library for making HTTP requests.
  Install it using: pip install requests
"""

import requests
import time
import sys

# Define the base URL for the UniProt REST API
UNIPROT_API_URL = "https://rest.uniprot.org/uniprotkb/search"

# List of 48 official HGNC gene symbols for human nuclear receptors
# Sourced from the HUGO Gene Nomenclature Committee (HGNC) database.
# See: https://www.genenames.org/data/genegroup/#!/group/471
HUMAN_NUCLEAR_RECEPTOR_GENES = sorted([
    "AR", "ESR1", "ESR2", "ESRRA", "ESRRB", "ESRRG", "HNF4A", "HNF4G",
    "NR0B1", "NR0B2", "NR1D1", "NR1D2", "NR1H2", "NR1H3", "NR1H4", "NR1I2",
    "NR1I3", "VDR", "NR2C1", "NR2C2", "NR2E1", "NR2E3", "NR2F1", "NR2F2",
    "NR2F6", "NR3C1", "NR3C2", "PGR", "NR4A1", "NR4A2", "NR4A3", "NR5A1",
    "NR5A2", "NR6A1", "PPARA", "PPARD", "PPARG", "RARA", "RARB", "RARG",
    "RORA", "RORB", "RORC", "RXRA", "RXRB", "RXRG", "THRA", "THRB"
])

def get_uniprot_id(gene_symbol):
    """
    Queries the UniProt API to find the reviewed UniProt ID for a given
    gene symbol in humans (organism ID 9606).

    Args:
        gene_symbol (str): The official HGNC gene symbol.

    Returns:
        str: The UniProt accession ID, or None if not found or an error occurs.
    """
    params = {
        'query': f'(gene:{gene_symbol}) AND (organism_id:9606) AND (reviewed:true)',
        'format': 'json',
        'fields': 'accession',
        'size': 1  # We only need the top result
    }

    try:
        response = requests.get(UNIPROT_API_URL, params=params, timeout=15)
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx or 5xx)

        data = response.json()
        if data.get('results'):
            # Extract the primary accession number from the first result
            return data['results'][0]['primaryAccession']
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while fetching data for {gene_symbol}: {e}", file=sys.stderr)
        return None
    except Exception as e:
        print(f"An unexpected error occurred for {gene_symbol}: {e}", file=sys.stderr)
        return None


def main():
    """
    Main function to orchestrate fetching of UniProt IDs and saving them
    to a file.
    """
    print("Fetching UniProt IDs for 48 human nuclear receptors...\n")
    
    receptor_uniprot_map = {}
    not_found_genes = []
    
    total_genes = len(HUMAN_NUCLEAR_RECEPTOR_GENES)

    for i, gene in enumerate(HUMAN_NUCLEAR_RECEPTOR_GENES):
        print(f"[{i+1}/{total_genes}] Searching for gene: {gene}...")
        uniprot_id = get_uniprot_id(gene)

        if uniprot_id:
            receptor_uniprot_map[gene] = uniprot_id
        else:
            print(f"  -> UniProt ID not found for {gene}")
            not_found_genes.append(gene)

        # Be polite to the API server by waiting briefly between requests
        time.sleep(0.5)

    # --- Save results to a file ---
    output_filename = "nuclear_receptor_uniprot_ids.txt"
    print(f"\nFetching complete. Writing results to '{output_filename}'...")

    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write("--- Human Nuclear Receptor UniProt ID Report ---\n")
            f.write(f"Successfully found IDs for {len(receptor_uniprot_map)} out of {total_genes} receptors.\n\n")

            # Write the found mappings to the file
            if receptor_uniprot_map:
                f.write("Gene Symbol -> UniProt ID\n")
                f.write("--------------------------\n")
                for gene, uid in receptor_uniprot_map.items():
                    f.write(f"{gene:<12} -> {uid}\n")

            # Write any genes that were not found to the file
            if not_found_genes:
                f.write("\nCould not find UniProt IDs for the following genes:\n")
                for gene in not_found_genes:
                    f.write(f"- {gene}\n")

            # Write the final list of unique UniProt IDs to the file
            unique_ids = sorted(list(set(receptor_uniprot_map.values())))
            if unique_ids:
                f.write("\n--- Unique UniProt IDs Collected ---\n")
                f.write(", ".join(unique_ids) + "\n")
                f.write(f"\nTotal unique IDs: {len(unique_ids)}\n")
        
        print(f"Successfully saved results to '{output_filename}'")
    except IOError as e:
        print(f"Error: Could not write to file '{output_filename}': {e}", file=sys.stderr)


if __name__ == "__main__":
    main()



Fetching UniProt IDs for 48 human nuclear receptors...

[1/48] Searching for gene: AR...
[2/48] Searching for gene: ESR1...
[3/48] Searching for gene: ESR2...
[4/48] Searching for gene: ESRRA...
[5/48] Searching for gene: ESRRB...
[6/48] Searching for gene: ESRRG...
[7/48] Searching for gene: HNF4A...
[8/48] Searching for gene: HNF4G...
[9/48] Searching for gene: NR0B1...
[10/48] Searching for gene: NR0B2...
[11/48] Searching for gene: NR1D1...
[12/48] Searching for gene: NR1D2...
[13/48] Searching for gene: NR1H2...
[14/48] Searching for gene: NR1H3...
[15/48] Searching for gene: NR1H4...
[16/48] Searching for gene: NR1I2...
[17/48] Searching for gene: NR1I3...
[18/48] Searching for gene: NR2C1...
[19/48] Searching for gene: NR2C2...
[20/48] Searching for gene: NR2E1...
[21/48] Searching for gene: NR2E3...
[22/48] Searching for gene: NR2F1...
[23/48] Searching for gene: NR2F2...
[24/48] Searching for gene: NR2F6...
[25/48] Searching for gene: NR3C1...
[26/48] Searching for gene: NR3C2

In [ ]:
def get_protein_details(uniprot_id):
    """
    Retrieves detailed information for a given UniProt accession ID.

    Args:
        uniprot_id (str): The UniProt accession ID.

    Returns:
        dict: A dictionary containing the protein's name and sequence length,
              or None if an error occurs.
    """
    # Correct URL format to fetch by accession is .../uniprotkb/{accession}
    details_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}"
    params = {'format': 'json'}
    try:
        response = requests.get(details_url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        name = data.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A')
        length = data.get('sequence', {}).get('length', 0)
        return {'name': name, 'length': length}
    except requests.exceptions.RequestException as e:
        print(f"Error fetching details for {uniprot_id}: {e}", file=sys.stderr)
        return None

def fetch_all_uniprot_ids(genes):
    """
    Fetches UniProt IDs for a list of gene symbols.

    Args:
        genes (list): A list of gene symbols.

    Returns:
        tuple: A tuple containing (receptor_uniprot_map, not_found_genes).
    """
    print("Step 1: Fetching UniProt IDs for human nuclear receptors...\n")
    receptor_uniprot_map = {}
    not_found_genes = []
    total_genes = len(genes)
    
    for i, gene in enumerate(genes):
        print(f"[{i+1}/{total_genes}] Searching for gene: {gene}...")
        uniprot_id = get_uniprot_id(gene)
        if uniprot_id:
            receptor_uniprot_map[gene] = uniprot_id
        else:
            print(f"  -> UniProt ID not found for {gene}")
            not_found_genes.append(gene)
        time.sleep(0.5) # Be polite to the API
    return receptor_uniprot_map, not_found_genes

def analyze_proteins(receptor_uniprot_map, min_length):
    """
    Analyzes proteins based on their length after fetching details.

    Args:
        receptor_uniprot_map (dict): Map of gene symbols to UniProt IDs.
        min_length (int): The minimum amino acid length to be considered a receptor.

    Returns:
        list: A list of dictionaries with analysis results.
    """
    print("\nStep 2: Retrieving protein details and applying criteria...")
    analysis_results = []
    unique_ids = sorted(list(set(receptor_uniprot_map.values())))
    total_ids = len(unique_ids)

    for i, uniprot_id in enumerate(unique_ids):
        associated_genes = [g for g, u in receptor_uniprot_map.items() if u == uniprot_id]
        gene_str = ", ".join(associated_genes)
        print(f"[{i+1}/{total_ids}] Analyzing {uniprot_id} (Gene: {gene_str})...")

        details = get_protein_details(uniprot_id)
        time.sleep(0.5) # Be polite to the API
        
        if details:
            is_receptor = details['length'] > min_length
            analysis_results.append({
                'gene': gene_str,
                'uniprot_id': uniprot_id,
                'name': details['name'],
                'length': details['length'],
                'is_receptor': is_receptor,
                'status': 'OK'
            })
        else:
             analysis_results.append({
                'gene': gene_str,
                'uniprot_id': uniprot_id,
                'status': 'Details fetch failed'
            })
    return analysis_results

def write_report(filename, results, not_found, min_length):
    """
    Writes the final analysis report to a text file.

    Args:
        filename (str): The name of the output file.
        results (list): The list of analysis result dictionaries.
        not_found (list): A list of genes for which no UniProt ID was found.
        min_length (int): The minimum length criterion used in the analysis.
    """
    print(f"\nStep 3: Writing analysis report to '{filename}'...")
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("--- Human Nuclear Receptor Polypeptide Analysis ---\n\n")
            f.write(f"Criterion: Considered a nuclear receptor if amino acid count > {min_length}.\n\n")
            f.write("--- Analysis Results ---\n")
            f.write(f"{'Gene(s)':<15} | {'UniProt ID':<12} | {'Length':<8} | {'Meets Criteria?':<18} | {'Protein Name'}\n")
            f.write("-" * 90 + "\n")

            for result in sorted(results, key=lambda x: x['gene']):
                if result['status'] == 'OK':
                    meets_criteria_str = 'Yes' if result['is_receptor'] else 'No'
                    f.write(f"{result['gene']:<15} | {result['uniprot_id']:<12} | {result['length']:<8} | {meets_criteria_str:<18} | {result['name']}\n")
                else:
                    f.write(f"{result['gene']:<15} | {result['uniprot_id']:<12} | {'N/A':<8} | {'N/A':<18} | {result['status']}\n")

            if not_found:
                f.write("\n--- Genes Without UniProt IDs ---\n")
                for gene in not_found:
                    f.write(f"- {gene}\n")
        
        print(f"Successfully saved report to '{filename}'")
    except IOError as e:
        print(f"Error: Could not write to file '{filename}': {e}", file=sys.stderr)

def main():
    """
    Main function to orchestrate fetching, analysis, and saving of results.
    """
    polypeptide_min_length = 50
    output_filename = "nuclear_receptor_analysis.txt"

    # Step 1: Fetch all the UniProt IDs
    id_map, not_found = fetch_all_uniprot_ids(HUMAN_NUCLEAR_RECEPTOR_GENES)
    
    # Step 2: Analyze the proteins based on the fetched IDs
    analysis_results = analyze_proteins(id_map, polypeptide_min_length)
    
    # Step 3: Write the results to a report file
    write_report(output_filename, analysis_results, not_found, polypeptide_min_length)

if __name__ == "__main__":
    main()

Step 1: Fetching UniProt IDs for human nuclear receptors...

[1/48] Searching for gene: AR...
[2/48] Searching for gene: ESR1...
[3/48] Searching for gene: ESR2...
[4/48] Searching for gene: ESRRA...
[5/48] Searching for gene: ESRRB...
[6/48] Searching for gene: ESRRG...
[7/48] Searching for gene: HNF4A...
[8/48] Searching for gene: HNF4G...
[9/48] Searching for gene: NR0B1...
[10/48] Searching for gene: NR0B2...
[11/48] Searching for gene: NR1D1...
[12/48] Searching for gene: NR1D2...
[13/48] Searching for gene: NR1H2...
[14/48] Searching for gene: NR1H3...
[15/48] Searching for gene: NR1H4...
[16/48] Searching for gene: NR1I2...
[17/48] Searching for gene: NR1I3...
[18/48] Searching for gene: NR2C1...
[19/48] Searching for gene: NR2C2...
[20/48] Searching for gene: NR2E1...
[21/48] Searching for gene: NR2E3...
[22/48] Searching for gene: NR2F1...
[23/48] Searching for gene: NR2F2...
[24/48] Searching for gene: NR2F6...
[25/48] Searching for gene: NR3C1...
[26/48] Searching for gene: 

In [4]:
# -*- coding: utf-8 -*-
"""
This script fetches detailed information for the 48 known human nuclear
receptors from UniProt. It identifies and extracts the Ligand-Binding Domain
(LBD) sequence for each receptor while also checking for the presence of
Zinc finger domains, which are characteristic of the DNA-Binding Domain (DBD).

It saves a summary report and a FASTA file containing the LBD sequences,
flagging any receptors where the LBD may be part of or overlap with a DBD.
"""

import requests
import time
import sys
import textwrap

# Define the base URL for the UniProt REST API
UNIPROT_API_URL = "https://rest.uniprot.org/uniprotkb"

# List of 48 official HGNC gene symbols for human nuclear receptors
HUMAN_NUCLEAR_RECEPTOR_GENES = sorted([
    "AR", "ESR1", "ESR2", "ESRRA", "ESRRB", "ESRRG", "HNF4A", "HNF4G",
    "NR0B1", "NR0B2", "NR1D1", "NR1D2", "NR1H2", "NR1H3", "NR1H4", "NR1I2",
    "NR1I3", "VDR", "NR2C1", "NR2C2", "NR2E1", "NR2E3", "NR2F1", "NR2F2",
    "NR2F6", "NR3C1", "NR3C2", "PGR", "NR4A1", "NR4A2", "NR4A3", "NR5A1",
    "NR5A2", "NR6A1", "PPARA", "PPARD", "PPARG", "RARA", "RARB", "RARG",
    "RORA", "RORB", "RORC", "RXRA", "RXRB", "RXRG", "THRA", "THRB"
])

def get_uniprot_id(gene_symbol):
    """Queries UniProt to find the reviewed UniProt ID for a gene symbol."""
    search_url = f"{UNIPROT_API_URL}/search"
    params = {
        'query': f'(gene:{gene_symbol}) AND (organism_id:9606) AND (reviewed:true)',
        'format': 'json',
        'fields': 'accession',
        'size': 1
    }
    try:
        response = requests.get(search_url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()
        return data['results'][0]['primaryAccession'] if data.get('results') else None
    except requests.exceptions.RequestException as e:
        print(f"Error fetching ID for {gene_symbol}: {e}", file=sys.stderr)
        return None

def get_and_extract_lbd_sequence(uniprot_id):
    """
    Retrieves protein details, extracts the LBD, and checks for Zinc fingers.
    """
    details_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}"
    params = {
        'format': 'json',
        'fields': 'protein_name,sequence,ft_domain,ft_zn_fing'
    }
    try:
        response = requests.get(details_url, params=params, timeout=15)
        response.raise_for_status()
        data = response.json()

        name = data.get('proteinDescription', {}).get('recommendedName', {}).get('fullName', {}).get('value', 'N/A')
        original_sequence = data.get('sequence', {}).get('value', '')
        original_length = data.get('sequence', {}).get('length', 0)
        
        lbd_info = {'sequence': "", 'location': "Not found", 'found': False, 'start': -1, 'end': -1}
        has_zn_finger = False

        # Search for the Ligand-binding domain in the features
        for feature in data.get('features', []):
            # --- FIX: Handle cases where description is a string or a dict ---
            description_obj = feature.get('description', {})
            description_text = ""
            if isinstance(description_obj, dict):
                description_text = description_obj.get('value', '')
            elif isinstance(description_obj, str):
                description_text = description_obj

            if feature.get('type') == 'Domain' and 'Ligand-binding' in description_text:
                location = feature.get('location', {})
                start = location.get('start', {}).get('value')
                end = location.get('end', {}).get('value')

                if start and end:
                    lbd_info['location'] = f"{start}-{end}"
                    lbd_info['found'] = True
                    lbd_info['start'] = start
                    lbd_info['end'] = end
                    lbd_info['sequence'] = original_sequence[start-1:end]
                    break 

        # Check for Zinc finger domains
        for feature in data.get('features', []):
             if feature.get('type') == 'Zinc finger':
                 has_zn_finger = True
                 break

        return {
            'name': name,
            'original_length': original_length,
            'lbd_sequence': lbd_info['sequence'],
            'lbd_found': lbd_info['found'],
            'lbd_location': lbd_info['location'],
            'has_zn_finger': has_zn_finger
        }

    except requests.exceptions.RequestException as e:
        print(f"Error fetching details for {uniprot_id}: {e}", file=sys.stderr)
        return None

def fetch_all_uniprot_ids(genes):
    """Fetches UniProt IDs for a list of gene symbols."""
    print("Step 1: Fetching UniProt IDs for human nuclear receptors...\n")
    receptor_uniprot_map = {}
    for i, gene in enumerate(genes):
        print(f"[{i+1}/{len(genes)}] Searching for gene: {gene}...")
        uniprot_id = get_uniprot_id(gene)
        if uniprot_id:
            receptor_uniprot_map[gene] = uniprot_id
        else:
            print(f"  -> UniProt ID not found for {gene}")
        time.sleep(0.5) # Be respectful to the API server
    return receptor_uniprot_map

def analyze_and_extract_lbds(receptor_uniprot_map):
    """Analyzes proteins to extract the Ligand-Binding Domain."""
    print("\nStep 2: Retrieving protein details and extracting LBD sequences...")
    analysis_results = []
    unique_ids = sorted(list(set(receptor_uniprot_map.values())))
    
    for i, uniprot_id in enumerate(unique_ids):
        associated_genes = [g for g, u in receptor_uniprot_map.items() if u == uniprot_id]
        gene_str = ", ".join(associated_genes)
        print(f"[{i+1}/{len(unique_ids)}] Analyzing {uniprot_id} (Gene: {gene_str})...")

        details = get_and_extract_lbd_sequence(uniprot_id)
        time.sleep(0.5)
        
        if details:
            details.update({'gene': gene_str, 'uniprot_id': uniprot_id, 'status': 'OK'})
            analysis_results.append(details)
        else:
             analysis_results.append({'gene': gene_str, 'uniprot_id': uniprot_id, 'status': 'Details fetch failed'})
    return analysis_results

def write_summary_report(filename, results):
    """Writes the LBD analysis summary report to a text file."""
    print(f"\nStep 3: Writing LBD analysis report to '{filename}'...")
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("--- Human Nuclear Receptor LBD Extraction Analysis ---\n\n")
        f.write("Action: Extracted the annotated 'Ligand-binding domain' from each sequence.\n")
        f.write("ZN Finger Check: Indicates if a Zinc finger (characteristic of a DBD) was annotated.\n\n")
        header = f"{'Gene(s)':<15} | {'UniProt ID':<12} | {'Has ZN-Finger?':<15} | {'LBD Found?':<11} | {'LBD Location':<12} | {'LBD Len':<10} | {'Protein Name'}\n"
        f.write(header)
        f.write("-" * (len(header) + 5) + "\n")

        for result in sorted(results, key=lambda x: x['gene']):
            if result['status'] == 'OK':
                zn_finger_str = 'Yes' if result['has_zn_finger'] else 'No'
                lbd_found_str = 'Yes' if result['lbd_found'] else 'No'
                lbd_len = len(result['lbd_sequence'])
                f.write(f"{result['gene']:<15} | {result['uniprot_id']:<12} | {zn_finger_str:<15} | {lbd_found_str:<11} | {result['lbd_location']:<12} | {lbd_len:<10} | {result['name']}\n")
            else:
                f.write(f"{result['gene']:<15} | {result['uniprot_id']:<12} | {'N/A':<15} | {'N/A':<11} | {'N/A':<12} | {'N/A':<10} | {result['status']}\n")
    print(f"Successfully saved report to '{filename}'")

def write_fasta_file(filename, results):
    """Writes the extracted LBD sequences to a FASTA format file."""
    print(f"Step 4: Writing LBD sequences to FASTA file '{filename}'...")
    with open(filename, 'w', encoding='utf-8') as f:
        for result in sorted(results, key=lambda x: x['gene']):
            if result['status'] == 'OK' and result['lbd_found']:
                zn_warning = " [WARNING: ZN-FINGER DOMAIN PRESENT]" if result['has_zn_finger'] else ""
                header = f">{result['uniprot_id']}|{result['gene']} {result['name']} [LBD extracted: {result['lbd_location']}]{zn_warning}"
                f.write(header + "\n")
                wrapped_sequence = textwrap.fill(result['lbd_sequence'], 80)
                f.write(wrapped_sequence + "\n")
    print(f"Successfully saved FASTA file to '{filename}'")

def main():
    """Main function to orchestrate the workflow."""
    summary_filename = "nuclear_receptor_LBD_analysis.txt"
    fasta_filename = "nuclear_receptor_LBDs.fasta"

    id_map = fetch_all_uniprot_ids(HUMAN_NUCLEAR_RECEPTOR_GENES)
    analysis_results = analyze_and_extract_lbds(id_map)
    write_summary_report(summary_filename, analysis_results)
    write_fasta_file(fasta_filename, analysis_results)

if __name__ == "__main__":
    main()



Step 1: Fetching UniProt IDs for human nuclear receptors...

[1/48] Searching for gene: AR...
[2/48] Searching for gene: ESR1...
[3/48] Searching for gene: ESR2...
[4/48] Searching for gene: ESRRA...
[5/48] Searching for gene: ESRRB...
[6/48] Searching for gene: ESRRG...
[7/48] Searching for gene: HNF4A...
[8/48] Searching for gene: HNF4G...
[9/48] Searching for gene: NR0B1...
[10/48] Searching for gene: NR0B2...
[11/48] Searching for gene: NR1D1...
[12/48] Searching for gene: NR1D2...
[13/48] Searching for gene: NR1H2...
[14/48] Searching for gene: NR1H3...
[15/48] Searching for gene: NR1H4...
[16/48] Searching for gene: NR1I2...
[17/48] Searching for gene: NR1I3...
[18/48] Searching for gene: NR2C1...
[19/48] Searching for gene: NR2C2...
[20/48] Searching for gene: NR2E1...
[21/48] Searching for gene: NR2E3...
[22/48] Searching for gene: NR2F1...
[23/48] Searching for gene: NR2F2...
[24/48] Searching for gene: NR2F6...
[25/48] Searching for gene: NR3C1...
[26/48] Searching for gene: 

In [5]:
import os
import requests
import time
import xml.etree.ElementTree as ET

# --- Configuration ---
TARGET_UNIPROT_ID = "P11473"  # UniProt ID for human Vitamin D Receptor (VDR)
DATA_DIRECTORY = "vdr_structures"
PDB_SEARCH_URL = "https://search.rcsb.org/rcsbsearch/v2/query"
PDB_DOWNLOAD_URL = "https://files.rcsb.org/download/"

def fetch_pdb_ids_for_uniprot(uniprot_id):
    """Fetches all PDB IDs associated with a single UniProt ID."""
    print(f"Fetching all PDB IDs for UniProt ID: {uniprot_id}...")
    query = {
        "query": {
            "type": "terminal",
            "service": "text",
            "parameters": {
                "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
                "operator": "exact_match",
                "value": uniprot_id
            }
        },
        "return_type": "entry",
        "request_options": {"return_all_hits": True}
    }
    response = requests.post(PDB_SEARCH_URL, json=query)
    if response.status_code == 200:
        pdb_ids = [item["identifier"] for item in response.json()["result_set"]]
        print(f"Found {len(pdb_ids)} associated PDB structures.")
        return pdb_ids
    else:
        print(f"Error fetching PDB IDs: {response.status_code}")
        return []

def download_pdb_xml(pdb_id, directory):
    """Downloads a single PDB structure in PDBML/XML format."""
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    url = f"{PDB_DOWNLOAD_URL}{pdb_id}.xml"
    filepath = os.path.join(directory, f"{pdb_id}.xml")

    if os.path.exists(filepath):
        # print(f"File {pdb_id}.xml already exists. Skipping download.")
        return True

    try:
        response = requests.get(url, timeout=20)
        response.raise_for_status()
        with open(filepath, 'wb') as f:
            f.write(response.content)
        return True
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {pdb_id}: {e}")
        return False

def contains_zinc(filepath):
    """
    Parses a PDBML/XML file to check for the presence of Zinc atoms.
    Returns True if a ZN atom is found, False otherwise.
    """
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        ns = {'pdbx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}
        
        # Efficiently search for any atom_site with a type_symbol of 'ZN'
        zn_atom = root.find('.//pdbx:atom_site/pdbx:type_symbol[.="ZN"]', ns)
        
        return zn_atom is not None
    except ET.ParseError:
        print(f"Warning: Could not parse {os.path.basename(filepath)}")
        return False # Treat unparseable files as not containing zinc for this purpose

def main():
    """Main function to run the VDR validation workflow."""
    
    # Step 1: Get all PDB IDs for VDR
    vdr_pdb_ids = fetch_pdb_ids_for_uniprot(TARGET_UNIPROT_ID)
    if not vdr_pdb_ids:
        return

    # Step 2: Download all the structure files
    print(f"\nDownloading {len(vdr_pdb_ids)} XML files to '{DATA_DIRECTORY}' directory...")
    for i, pdb_id in enumerate(vdr_pdb_ids):
        print(f"[{i+1}/{len(vdr_pdb_ids)}] Downloading {pdb_id}...")
        download_pdb_xml(pdb_id, DATA_DIRECTORY)
        time.sleep(0.1) # Be respectful to the server

    # Step 3: Classify each structure
    print("\nClassifying structures based on Zinc presence...")
    dbd_structures = []
    lbd_structures = []

    for pdb_id in vdr_pdb_ids:
        filepath = os.path.join(DATA_DIRECTORY, f"{pdb_id}.xml")
        if os.path.exists(filepath):
            if contains_zinc(filepath):
                dbd_structures.append(pdb_id)
            else:
                lbd_structures.append(pdb_id)

    # Step 4: Report the findings
    print("\n" + "="*50)
    print("=== VDR Structure Classification Results ===")
    print("="*50)
    print(f"Total Structures Analyzed: {len(vdr_pdb_ids)}")
    print(f"\nStructures Classified as DBD (contain Zinc): {len(dbd_structures)}")
    print("-" * 40)
    print(", ".join(sorted(dbd_structures)))
    
    print(f"\nStructures Classified as LBD (do not contain Zinc): {len(lbd_structures)}")
    print("-" * 40)
    print(", ".join(sorted(lbd_structures)))
    print("\n" + "="*50)

if __name__ == "__main__":
    main()


Fetching all PDB IDs for UniProt ID: P11473...
Found 52 associated PDB structures.

[1/52] Downloading 1DB1...
[2/52] Downloading 1IE8...
[3/52] Downloading 1IE9...
[4/52] Downloading 1KB2...
[5/52] Downloading 1KB4...
[6/52] Downloading 1KB6...
[7/52] Downloading 1S0Z...
[8/52] Downloading 1S19...
[9/52] Downloading 1TXI...
[10/52] Downloading 1YNW...
[11/52] Downloading 2HAM...
[12/52] Downloading 2HAR...
[13/52] Downloading 2HAS...
[14/52] Downloading 2HB7...
[15/52] Downloading 2HB8...
[16/52] Downloading 3A2I...
[17/52] Downloading 3A2J...
[18/52] Downloading 3A3Z...
[19/52] Downloading 3A40...
[20/52] Downloading 3A78...
[21/52] Downloading 3AUQ...
[22/52] Downloading 3AUR...
[23/52] Downloading 3AX8...
[24/52] Downloading 3AZ1...
[25/52] Downloading 3AZ2...
[26/52] Downloading 3AZ3...
[27/52] Downloading 3B0T...
[28/52] Downloading 3CS4...
[29/52] Downloading 3CS6...
[30/52] Downloading 3KPZ...
[31/52] Downloading 3M7R...
[32/52] Downloading 3OGT...
[33/52] Downloading 3P8X...
[

In [6]:
import os
import requests
import time
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor

# --- Configuration ---
# Using the full list of 48 UniProt IDs for all human NRs
NR_UNIPROT_IDS = sorted([
    "P10275", "P03372", "Q92731", "P11474", "O95718", "P62508", "P41235", 
    "Q14541", "P51843", "Q15466", "P20393", "Q14995", "P55055", "Q13133", 
    "Q96RI1", "O75469", "Q14994", "P11473", "P13056", "P49116", "Q9Y466", 
    "Q9Y5X4", "P10589", "P24468", "P10588", "P04150", "P08235", "P06401", 
    "P22736", "P43354", "Q92570", "Q13285", "O00482", "Q15406", "Q07869", 
    "Q03181", "P37231", "P10276", "P10826", "P13631", "P35398", "Q92753", 
    "P51449", "P19793", "P28702", "P48443", "P10827", "P10828"
])
# Main directory to store all downloaded structures
BASE_DATA_DIRECTORY = "all_nr_structures"
PDB_SEARCH_URL = "https://search.rcsb.org/rcsbsearch/v2/query"
PDB_DOWNLOAD_URL = "https://files.rcsb.org/download/"

def fetch_pdb_ids_for_uniprot(uniprot_id):
    """Fetches all PDB IDs associated with a single UniProt ID."""
    print(f"Fetching PDB IDs for UniProt ID: {uniprot_id}...")
    query = {
        "query": {
            "type": "terminal",
            "service": "text",
            "parameters": {
                "attribute": "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
                "operator": "exact_match",
                "value": uniprot_id
            }
        },
        "return_type": "entry",
        "request_options": {"return_all_hits": True}
    }
    try:
        response = requests.post(PDB_SEARCH_URL, json=query, timeout=30)
        response.raise_for_status()
        pdb_ids = [item["identifier"] for item in response.json()["result_set"]]
        print(f" -> Found {len(pdb_ids)} associated PDB structures.")
        return uniprot_id, pdb_ids
    except requests.exceptions.RequestException as e:
        print(f" -> Error fetching PDB IDs for {uniprot_id}: {e}")
        return uniprot_id, []

def download_pdb_xml(args):
    """Downloads a single PDB structure in PDBML/XML format."""
    pdb_id, directory = args
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)
    
    filepath = os.path.join(directory, f"{pdb_id}.xml")
    if os.path.exists(filepath): return pdb_id, True # Skip if already exists

    try:
        url = f"{PDB_DOWNLOAD_URL}{pdb_id}.xml"
        response = requests.get(url, timeout=20)
        response.raise_for_status()
        with open(filepath, 'wb') as f: f.write(response.content)
        return pdb_id, True
    except requests.exceptions.RequestException:
        return pdb_id, False # Return False on failure

def contains_zinc(filepath):
    """Parses a PDBML/XML file to check for the presence of Zinc atoms."""
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        ns = {'pdbx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}
        zn_atom = root.find('.//pdbx:atom_site/pdbx:type_symbol[.="ZN"]', ns)
        return zn_atom is not None
    except ET.ParseError:
        return False

def main():
    """Main function to run the full filtering workflow."""
    
    # Step 1: Get all PDB IDs for all 48 NRs
    print("--- Step 1: Fetching PDB IDs for all 48 NRs ---")
    all_pdb_ids_map = {}
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = executor.map(fetch_pdb_ids_for_uniprot, NR_UNIPROT_IDS)
        for uniprot_id, pdb_ids in results:
            all_pdb_ids_map[uniprot_id] = pdb_ids
    
    # Step 2: Download all structure files, organized by UniProt ID
    print("\n--- Step 2: Downloading all unique structure files ---")
    # Create a flat list of all unique PDBs to download
    unique_pdb_ids = set()
    for ids in all_pdb_ids_map.values():
        unique_pdb_ids.update(ids)
        
    download_tasks = []
    for uniprot_id, pdb_ids in all_pdb_ids_map.items():
        # Create a specific directory for each NR
        nr_directory = os.path.join(BASE_DATA_DIRECTORY, uniprot_id)
        for pdb_id in pdb_ids:
            download_tasks.append((pdb_id, nr_directory))

    print(f"Downloading {len(unique_pdb_ids)} unique structures into organized folders...")
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Use a simple progress indicator
        for i, (pdb_id, success) in enumerate(executor.map(download_pdb_xml, download_tasks)):
            sys.stdout.write(f"\rDownloaded {i+1}/{len(download_tasks)}...")
            sys.stdout.flush()
    print("\nDownload complete.")

    # Step 3: Classify each structure and compile a final report
    print("\n--- Step 3: Classifying all structures and generating report ---")
    report_data = []
    for uniprot_id in NR_UNIPROT_IDS:
        pdb_ids = all_pdb_ids_map.get(uniprot_id, [])
        nr_directory = os.path.join(BASE_DATA_DIRECTORY, uniprot_id)
        
        dbd_count = 0
        lbd_count = 0
        for pdb_id in pdb_ids:
            filepath = os.path.join(nr_directory, f"{pdb_id}.xml")
            if os.path.exists(filepath):
                if contains_zinc(filepath):
                    dbd_count += 1
                else:
                    lbd_count += 1
        
        report_data.append({
            "uniprot_id": uniprot_id,
            "total": len(pdb_ids),
            "dbd": dbd_count,
            "lbd": lbd_count
        })

    # Step 4: Write the final report
    report_filename = "full_nr_structural_classification_report.txt"
    print(f"\nWriting final report to '{report_filename}'...")
    with open(report_filename, 'w') as f:
        f.write("="*70 + "\n")
        f.write("=== Full Nuclear Receptor Structural Classification Report ===\n")
        f.write("="*70 + "\n\n")
        f.write("Classification is based on the presence of atomic Zinc (ZN) in the PDB file.\n")
        f.write("Structures with Zinc are classified as DBD; those without are LBD candidates.\n\n")
        
        header = f"{'UniProt ID':<12} | {'Total Structures':<18} | {'DBD Count (w/ ZN)':<20} | {'LBD Count (no ZN)':<20}\n"
        f.write(header)
        f.write("-" * (len(header) + 5) + "\n")
        
        for item in sorted(report_data, key=lambda x: x['uniprot_id']):
            f.write(f"{item['uniprot_id']:<12} | {item['total']:<18} | {item['dbd']:<20} | {item['lbd']:<20}\n")
    
    print("Process complete.")

if __name__ == "__main__":
    main()


--- Step 1: Fetching PDB IDs for all 48 NRs ---
Fetching PDB IDs for UniProt ID: O00482...
Fetching PDB IDs for UniProt ID: O75469...
Fetching PDB IDs for UniProt ID: O95718...
Fetching PDB IDs for UniProt ID: P03372...
Fetching PDB IDs for UniProt ID: P04150...
 -> Found 74 associated PDB structures.
Fetching PDB IDs for UniProt ID: P06401...
 -> Found 57 associated PDB structures.
Fetching PDB IDs for UniProt ID: P08235...
 -> Found 28 associated PDB structures.
Fetching PDB IDs for UniProt ID: P10275...
 -> Found 474 associated PDB structures.
Fetching PDB IDs for UniProt ID: P10276...
 -> Found 20 associated PDB structures.
Fetching PDB IDs for UniProt ID: P10588...
 -> Found 3 associated PDB structures.
Fetching PDB IDs for UniProt ID: P10589...
 -> Found 94 associated PDB structures.
Fetching PDB IDs for UniProt ID: P10826...
 -> Found 29 associated PDB structures.
Fetching PDB IDs for UniProt ID: P10827...
 -> Found 14 associated PDB structures.
Fetching PDB IDs for UniProt ID: 

In [8]:
import os
import shutil
import xml.etree.ElementTree as ET

# --- Configuration ---
# The source directory where all structures were downloaded by the previous script.
SOURCE_DIRECTORY = "all_nr_structures"
# The destination directory that the next script ('filter_for_peptides.py') will use.
DESTINATION_DIRECTORY = "lbd_structures"

def contains_zinc(filepath: str) -> bool:
    """
    Parses a PDBML/XML file to check for the presence of Zinc atoms.
    Returns True if a ZN atom is found, False otherwise.
    """
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        ns = {'pdbx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}
        # Use a more efficient XPath to find the element
        zn_atom = root.find('.//pdbx:atom_site/pdbx:type_symbol[.="ZN"]', ns)
        return zn_atom is not None
    except ET.ParseError:
        print(f"  - Warning: Could not parse {os.path.basename(filepath)}, skipping.")
        return True # Treat unparseable files as if they have zinc to be safe

def main():
    """
    Main function to find all LBD structures (no zinc) and copy them
    to a new, clean directory for the next analysis step.
    """
    if not os.path.isdir(SOURCE_DIRECTORY):
        print(f"Error: Source directory '{SOURCE_DIRECTORY}' not found.")
        print("Please make sure you have run the 'filter_all_nr_structures.py' script first.")
        return

    # Create the destination directory if it doesn't exist
    if not os.path.exists(DESTINATION_DIRECTORY):
        print(f"Creating destination directory: '{DESTINATION_DIRECTORY}'")
        os.makedirs(DESTINATION_DIRECTORY)

    print(f"Scanning '{SOURCE_DIRECTORY}' for LBD structures (files without Zinc)...")
    
    lbd_files_found = 0
    total_files_scanned = 0

    # Walk through all the subdirectories (e.g., P10275, P11473, etc.)
    for root, _, files in os.walk(SOURCE_DIRECTORY):
        for filename in files:
            if filename.endswith(".xml"):
                total_files_scanned += 1
                filepath = os.path.join(root, filename)
                
                # If the file does NOT contain zinc, it's an LBD candidate
                if not contains_zinc(filepath):
                    lbd_files_found += 1
                    destination_path = os.path.join(DESTINATION_DIRECTORY, filename)
                    
                    # Copy the file to the new 'lbd_structures' folder
                    shutil.copy2(filepath, destination_path)
                    print(f"  -> Copying LBD file: {filename}")

    print("\n--- Process Complete ---")
    print(f"Scanned {total_files_scanned} total XML files.")
    print(f"Found and copied {lbd_files_found} LBD structures to the '{DESTINATION_DIRECTORY}' folder.")
    print("\nYou can now run the 'filter_for_peptides.py' script.")

if __name__ == "__main__":
    main()


Creating destination directory: 'lbd_structures'
Scanning 'all_nr_structures' for LBD structures (files without Zinc)...
  -> Copying LBD file: 2ZVT.xml
  -> Copying LBD file: 8HUP.xml
  -> Copying LBD file: 6DCU.xml
  -> Copying LBD file: 2HFP.xml
  -> Copying LBD file: 1ZEO.xml
  -> Copying LBD file: 8ZFO.xml
  -> Copying LBD file: 9GX1.xml
  -> Copying LBD file: 9GWH.xml
  -> Copying LBD file: 3VJH.xml
  -> Copying LBD file: 8REJ.xml
  -> Copying LBD file: 7CXF.xml
  -> Copying LBD file: 7SQB.xml
  -> Copying LBD file: 8B93.xml
  -> Copying LBD file: 8B92.xml
  -> Copying LBD file: 4CI5.xml
  -> Copying LBD file: 8B8W.xml
  -> Copying LBD file: 1NYX.xml
  -> Copying LBD file: 7CXG.xml
  -> Copying LBD file: 3OSI.xml
  -> Copying LBD file: 3D6D.xml
  -> Copying LBD file: 1RDT.xml
  -> Copying LBD file: 3VJI.xml
  -> Copying LBD file: 3ADX.xml
  -> Copying LBD file: 5UGM.xml
  -> Copying LBD file: 4OJ4.xml
  -> Copying LBD file: 9GWI.xml
  -> Copying LBD file: 8ADF.xml
  -> Copying LB

In [ ]:
import os
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from typing import List, Optional

# --- Configuration ---
MIN_RECEPTOR_LENGTH = 50
MAX_PEPTIDE_LENGTH = 49 # Anything less than 50

# --- EDIT THIS LINE ---
# Please replace this with the full path to the folder containing your LBD XML files.
# For example: "/Users/danamyktybek/Desktop/biol363_project/lbd_structures"
DATA_DIRECTORY = "/Users/danamyktybek/Desktop/biol363_project/biol363/lbd_structures" # Folder containing your LBD candidate XML files

@dataclass
class PolypeptideEntity:
    """A simple container for a unique polypeptide molecule."""
    entity_id: str
    sequence: str
    description: str

def extract_polypeptide_entities(root: ET.Element) -> List[PolypeptideEntity]:
    """
    Parses an XML root to find all unique polypeptide entities and their sequences.
    """
    ns = {'pdbx': 'http://pdbml.pdb.org/schema/pdbx-v50.xsd'}
    entities = []
    
    for poly_entity in root.findall('.//pdbx:entity_poly', ns):
        entity_type_tag = poly_entity.find('pdbx:type', ns)
        if entity_type_tag is None or entity_type_tag.text != 'polypeptide(L)':
            continue

        entity_id_tag = poly_entity.find('pdbx:entity_id', ns)
        seq_tag = poly_entity.find('pdbx:pdbx_seq_one_letter_code_can', ns)
        if seq_tag is None or not seq_tag.text:
            seq_tag = poly_entity.find('pdbx:pdbx_seq_one_letter_code', ns)

        # Entity must have an ID and a sequence to be valid
        if entity_id_tag is not None and entity_id_tag.text and seq_tag is not None and seq_tag.text:
            entity_id = entity_id_tag.text
            sequence = seq_tag.text.strip().replace('\n', '')
            
            # Find the corresponding description for this entity
            description = "N/A"
            entity_desc_tag = root.find(f'.//pdbx:entity[@id="{entity_id}"]/pdbx:pdbx_description', ns)
            if entity_desc_tag is not None and entity_desc_tag.text:
                description = entity_desc_tag.text.strip()
            
            entities.append(PolypeptideEntity(
                entity_id=entity_id,
                sequence=sequence,
                description=description
            ))
            
    return entities

def is_nr_peptide_complex(filepath: str) -> (bool, str):
    """
    Determines if a structure file represents an NR-LBD bound to a peptide.
    """
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
    except ET.ParseError:
        return False, "Rejected (XML Parse Error)"

    entities = extract_polypeptide_entities(root)

    # Rule 1: Must have at least two unique polypeptide entities
    if len(entities) < 2:
        return False, "Rejected (Fewer than 2 unique polypeptide entities)"

    # Rule 2 & 3: Check for one long chain (NR) and one short chain (peptide)
    has_receptor_candidate = any(len(e.sequence) >= MIN_RECEPTOR_LENGTH for e in entities)
    has_peptide_candidate = any(len(e.sequence) <= MAX_PEPTIDE_LENGTH for e in entities)

    if has_receptor_candidate and has_peptide_candidate:
        return True, "Passed (Contains at least one long chain and one short peptide)"
    elif not has_receptor_candidate:
        return False, "Rejected (No chain meets the minimum receptor length of 50 AAs)"
    else: # Has a receptor but no peptide
        return False, "Rejected (Contains a receptor, but no separate peptide chain)"

def main():
    """Main function to run the filtering workflow."""
    if not os.path.isdir(DATA_DIRECTORY) or DATA_DIRECTORY == "/path/to/your/lbd_structures":
        print(f"Error: The data directory has not been set.")
        print("Please edit the 'DATA_DIRECTORY' variable in the script to the full path of your 'lbd_structures' folder.")
        return

    xml_files = [f for f in os.listdir(DATA_DIRECTORY) if f.endswith('.xml')]
    if not xml_files:
        print(f"Error: No .xml files found in '{DATA_DIRECTORY}'.")
        return

    print(f"Screening {len(xml_files)} LBD candidate structures for peptide complexes...\n")
    
    complex_pdb_ids = []
    
    for filename in sorted(xml_files):
        pdb_id = filename.split('.')[0]
        filepath = os.path.join(DATA_DIRECTORY, filename)
        
        is_complex, reason = is_nr_peptide_complex(filepath)
        
        if is_complex:
            print(f"✅ {pdb_id}: {reason}")
            complex_pdb_ids.append(pdb_id)
        else:
            print(f"❌ {pdb_id}: {reason}")

    # Write the final report
    output_filename = "nr_peptide_complexes.txt"
    print(f"\n--- Filtering Complete ---")
    print(f"Found {len(complex_pdb_ids)} structures that are potential NR-peptide complexes.")
    
    with open(output_filename, 'w') as f:
        f.write("# PDB IDs of LBD structures classified as being in complex with a peptide ligand (<50 AAs)\n")
        for pdb_id in sorted(complex_pdb_ids):
            f.write(f"{pdb_id}\n")
            
    print(f"The final list has been saved to '{output_filename}'.")

if __name__ == "__main__":
    main()



Error: The data directory has not been set.
Please edit the 'DATA_DIRECTORY' variable in the script to the full path of your 'lbd_structures' folder.
